In [1]:
pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00


In [2]:
from finvizfinance.screener.overview import Overview
import yfinance as yf
import pandas as pd
import numpy as np

class QMJModel:
    def __init__(self):
        self.screener = Overview()

    def get_finviz_data(self):
        # Obtener datos de cada sector por separado y concatenar
        sectors = ['Technology', 'Financial', 'Healthcare', 'Consumer Defensive']
        all_data = []

        for sector in sectors:
            filters_dict = {
                'Exchange': 'NASDAQ',
                'Market Cap.': '+Mid (over $2bln)',
                'Average Volume': 'Over 500K',
                'Price': 'Over $10',
                'Sector': sector
            }
            self.screener.set_filter(filters_dict=filters_dict)
            sector_data = self.screener.screener_view()
            all_data.append(sector_data)

        return pd.concat(all_data, ignore_index=True)

    def get_yf_data(self, ticker):
        try:
            stock = yf.Ticker(ticker)
            info = stock.info

            return {
                'Ticker': ticker,
                'ROE': info.get('returnOnEquity', 0),
                'ROA': info.get('returnOnAssets', 0),
                'OperatingMargin': info.get('operatingMargins', 0),
                'Beta': info.get('beta', 1),
                'DebtToEquity': info.get('debtToEquity', 0)
            }
        except:
            return None

    def calculate_quality_score(self):
        df = self.get_finviz_data()

        detailed_data = []
        for ticker in df['Ticker']:
            data = self.get_yf_data(ticker)
            if data is not None:
                detailed_data.append(data)

        detailed_df = pd.DataFrame(detailed_data)
        df = df.merge(detailed_df, on='Ticker', how='inner')

        # Winsorización
        for col in ['ROE', 'ROA', 'OperatingMargin', 'Beta', 'DebtToEquity']:
            df[col] = df[col].clip(lower=df[col].quantile(0.05), upper=df[col].quantile(0.95))

        profitability = df[['ROE', 'ROA', 'OperatingMargin']].mean(axis=1)
        safety = -df[['Beta', 'DebtToEquity']].mean(axis=1)

        df['Profitability_Score'] = (profitability - profitability.mean()) / profitability.std()
        df['Safety_Score'] = (safety - safety.mean()) / safety.std()
        df['Quality_Score'] = (df['Profitability_Score'] + df['Safety_Score']) / 2

        return df.sort_values('Quality_Score', ascending=False)

    def get_qmj_portfolio(self, n_stocks=5):
        df = self.calculate_quality_score()

        if len(df) < n_stocks * 2:
            raise ValueError(f"Insufficient stocks ({len(df)}) for portfolio construction")

        high_quality = df.head(n_stocks)
        low_quality = df.tail(n_stocks)

        return {
            'long': high_quality[['Ticker', 'Company', 'Sector', 'Quality_Score', 'ROE', 'Beta']],
            'short': low_quality[['Ticker', 'Company', 'Sector', 'Quality_Score', 'ROE', 'Beta']],
            'qmj_spread': high_quality['Quality_Score'].mean() - low_quality['Quality_Score'].mean()
        }

# Uso
qmj = QMJModel()
portfolio = qmj.get_qmj_portfolio()
print("\nPortfolio Long (Alta Calidad):")
print(portfolio['long'])
print("\nPortfolio Short (Baja Calidad):")
print(portfolio['short'])
print("\nQMJ Spread:", portfolio['qmj_spread'])

/usr/local/lib/python3.10/dist-packages/finvizfinance/screener/base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)



Portfolio Long (Alta Calidad):
    Ticker                                Company      Sector  Quality_Score  \
224   CPRX           Catalyst Pharmaceuticals Inc  Healthcare       0.751529   
92    NVDA                            NVIDIA Corp  Technology       0.723426   
187   TBBK                            Bancorp Inc   Financial       0.723160   
33    CHKP  Check Point Software Technologies Ltd  Technology       0.720911   
180   QFIN               Qifu Technology Inc. ADR   Financial       0.696643   

         ROE   Beta  
224  0.28291  0.755  
92   0.49196  1.657  
187  0.25885  1.462  
33   0.29839  0.633  
180  0.21977  0.659  

Portfolio Short (Baja Calidad):
    Ticker                        Company      Sector  Quality_Score  \
274   RARE  Ultragenyx Pharmaceutical Inc  Healthcare      -1.810870   
247   INSM                     Insmed Inc  Healthcare      -2.094601   
20    ASTS            AST SpaceMobile Inc  Technology      -2.245687   
300   VRNA          Verona Pharma 

In [ ]:
from finvizfinance.screener.overview import Overview
import yfinance as yf
import pandas as pd
import numpy as np
import time

class QMJModel:
    def __init__(self):
        self.screener = Overview()

    def get_finviz_data(self):
        sectors = ['Technology', 'Financial', 'Healthcare', 'Consumer Defensive']
        all_data = []

        for sector in sectors:
            filters_dict = {
                'Exchange': 'NASDAQ',
                'Market Cap.': '+Mid (over $2bln)',
                'Average Volume': 'Over 500K',
                'Price': 'Over $10',
                'Sector': sector
            }
            self.screener.set_filter(filters_dict=filters_dict)
            sector_data = self.screener.screener_view()
            if not sector_data.empty:
                all_data.append(sector_data)

        return pd.concat(all_data, ignore_index=True)

    def get_yf_data(self, ticker):
        try:
            stock = yf.Ticker(ticker)
            info = stock.info

            # Obtener datos históricos
            hist = stock.history(period="3mo")

            # Calcular métricas de crecimiento
            if not hist.empty:
                returns = hist['Close'].pct_change()
                growth_rate = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
                volatility = returns.std() * np.sqrt(252)  # Anualizada
            else:
                growth_rate = 0
                volatility = 0

            time.sleep(0.1)  # Evitar límite de requests

            return {
                'Ticker': ticker,
                'ROE': info.get('returnOnEquity', 0),
                'ROA': info.get('returnOnAssets', 0),
                'OperatingMargin': info.get('operatingMargins', 0),
                'Beta': info.get('beta', 1),
                'DebtToEquity': info.get('debtToEquity', 0),
                'GrowthRate': growth_rate,
                'Volatility': volatility
            }
        except Exception as e:
            print(f"Error getting data for {ticker}: {str(e)}")
            return None

    def calculate_quality_score(self):
        df = self.get_finviz_data()

        detailed_data = []
        for ticker in df['Ticker']:
            data = self.get_yf_data(ticker)
            if data is not None:
                detailed_data.append(data)

        if not detailed_data:
            raise ValueError("No data available for analysis")

        detailed_df = pd.DataFrame(detailed_data)
        df = df.merge(detailed_df, on='Ticker', how='inner')

        # Winsorización
        for col in ['ROE', 'ROA', 'OperatingMargin', 'Beta', 'DebtToEquity', 'GrowthRate', 'Volatility']:
            df[col] = df[col].clip(lower=df[col].quantile(0.05), upper=df[col].quantile(0.95))

        profitability = df[['ROE', 'ROA', 'OperatingMargin']].mean(axis=1)
        growth = df['GrowthRate']
        safety = -(df[['Beta', 'DebtToEquity', 'Volatility']].mean(axis=1))

        df['Profitability_Score'] = (profitability - profitability.mean()) / profitability.std()
        df['Growth_Score'] = (growth - growth.mean()) / growth.std()
        df['Safety_Score'] = (safety - safety.mean()) / safety.std()

        df['Quality_Score'] = (df['Profitability_Score'] + df['Growth_Score'] + df['Safety_Score']) / 3

        return df.sort_values('Quality_Score', ascending=False)

    def get_qmj_portfolio(self, n_stocks=5):
        df = self.calculate_quality_score()

        if len(df) < n_stocks * 2:
            raise ValueError(f"Insufficient stocks ({len(df)}) for portfolio construction")

        high_quality = df.head(n_stocks)
        low_quality = df.tail(n_stocks)

        return {
            'long': high_quality[['Ticker', 'Company', 'Sector', 'Quality_Score', 'ROE', 'GrowthRate', 'Beta']],
            'short': low_quality[['Ticker', 'Company', 'Sector', 'Quality_Score', 'ROE', 'GrowthRate', 'Beta']],
            'qmj_spread': high_quality['Quality_Score'].mean() - low_quality['Quality_Score'].mean()
        }

# Uso
qmj = QMJModel()
portfolio = qmj.get_qmj_portfolio()
print("\nPortfolio Long (Alta Calidad):")
print(portfolio['long'])
print("\nPortfolio Short (Baja Calidad):")
print(portfolio['short'])
print("\nQMJ Spread:", portfolio['qmj_spread'])

/usr/local/lib/python3.10/dist-packages/finvizfinance/screener/base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)
ERROR:yfinance:TTAN: Period '3mo' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MASI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MASI&crumb=Edge%3A+Too+Many+Requests


Error getting data for MASI: Expecting value: line 1 column 1 (char 0)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MRNA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MRNA&crumb=Edge%3A+Too+Many+Requests


Error getting data for MRNA: Expecting value: line 1 column 1 (char 0)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MRUS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MRUS&crumb=Edge%3A+Too+Many+Requests


...

Error getting data for PPC: Expecting value: line 1 column 1 (char 0)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SFM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SFM&crumb=Edge%3A+Too+Many+Requests


Error getting data for SFM: Expecting value: line 1 column 1 (char 0)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SMPL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SMPL&crumb=Edge%3A+Too+Many+Requests


Error getting data for SMPL: Expecting value: line 1 column 1 (char 0)

Portfolio Long (Alta Calidad):
    Ticker                       Company      Sector  Quality_Score      ROE  \
104   PLTR     Palantir Technologies Inc  Technology       1.132484  0.12409   
163   GGAL  Grupo Financiero Galicia ADR   Financial       1.122054  0.35249   
121   SRAD           Sportradar Group AG  Technology       1.089137  0.06785   
9     ALAB               Astera Labs Inc  Technology       1.020084  0.00000   
106   PYCR                Paycor HCM Inc  Technology       0.999872 -0.03552   

     GrowthRate    Beta  
104   54.585164  2.0109  
163   44.786009  1.5910  
121   53.102339  2.0109  
9     55.447057  1.0000  
106   55.447057  0.4200  

Portfolio Short (Baja Calidad):
    Ticker                        Company      Sector  Quality_Score  \
20    ASTS            AST SpaceMobile Inc  Technology      -1.496155   
208   ARWR  Arrowhead Pharmaceuticals Inc  Healthcare      -1.571513   
248   IONS 